In [1]:
import pandas as pd
import nltk

In [2]:
data = pd.read_csv('/Users/gavin/documents/metis/coursework/project_4/dataframes/per_comment.csv').drop('Unnamed: 0', axis=1)

In [3]:
# Text preprocessing steps - remove numbers, captial letters and punctuation
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
to_string = lambda x: f'{x}'

data['text'] = data.text.map(to_string).map(alphanumeric).map(punc_lower)
data.head()

,url,url_time,date,speaker,time,text,datetime,week
0,https://www.rev.com/blog/transcripts/new-york-...,https://www.rev.com/transcript-editor/shared/0...,"Oct 26, 2020",Andrew Cuomo,00:02,people who are with me on the dais my far lef...,2020-10-26,44
1,https://www.rev.com/blog/transcripts/new-york-...,https://www.rev.com/transcript-editor/shared/0...,"Oct 26, 2020",Andrew Cuomo,01:03,okay today is day but i had a revelation o...,2020-10-26,44
2,https://www.rev.com/blog/transcripts/new-york-...,https://www.rev.com/transcript-editor/shared/0...,"Oct 26, 2020",Andrew Cuomo,02:31,we’re not going to control the pandemic all w...,2020-10-26,44
3,https://www.rev.com/blog/transcripts/new-york-...,https://www.rev.com/transcript-editor/shared/0...,"Oct 26, 2020",Andrew Cuomo,03:53,they’re watching what happened in china and it...,2020-10-26,44
4,https://www.rev.com/blog/transcripts/new-york-...,https://www.rev.com/transcript-editor/shared/0...,"Oct 26, 2020",Andrew Cuomo,05:07,it was never let’s keep the economy open vers...,2020-10-26,44


In [4]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [5]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(['hasn', 'covid','inaudible', 'way', 'different','today', 'mr', 'person', 'help', 'virus', 'going', 'like', 'people', 'said', 'know', 'doing', 'say','good', 'day', 'think', 'did', 'look', 'let', 'right', 've', 'i', 'll', 'new', 'city', 'york', 'state', 'm', 'don', 'come', 'time' 'inaudible','didn','yes','yeah','thing','sir','guys','crosstalk','hey' 'doesn','t','got','okay','hello', 'great', 'says', 'just', 'want','somebody', 'does','thank',  'week', 'make'])

In [6]:
def doc_matrix(text_list, vectorizer):
    vectorizer = vectorizer(stop_words=my_stop_words)
    vectors = vectorizer.fit_transform(text_list)
    return pd.DataFrame(vectors.toarray(), columns = vectorizer.get_feature_names())

In [7]:
corpus = [comment for comment in data['text']]

In [8]:
matrix = doc_matrix(corpus, TfidfVectorizer)
matrix.head()

,abandoned,abandonment,abate,abated,abatements,abating,abbott,abbreviated,abdicate,abdominal,...,zip,zippo,zithromax,zithromycin,zone,zones,zoom,zoomed,zucker,zuker
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.176231,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
tuple(matrix[2000].nlargest(15).index)

KeyError: 2000

In [10]:
len(matrix)

10474

In [11]:
# sklearn
from sklearn import datasets
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# logging (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(matrix)

LatentDirichletAllocation(random_state=0)

In [12]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = my_stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                ngram_range=(1,2),
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(corpus)
print(dtm_tf.shape)

(10474, 3955)


In [13]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(dtm_tfidf.shape)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1795: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


(10474, 3955)


In [14]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=9, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=9, random_state=0)

In [15]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

2020-11-09 20:29:08,223 : INFO : NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.033676  0.016549       1        1  23.574068
4      0.089973 -0.096428       2        1  22.329608
1      0.097088  0.022171       3        1  13.779723
6      0.072790  0.097151       4        1   9.869769
2      0.004177 -0.204226       5        1   7.459430
3      0.073989  0.089739       6        1   6.858433
7     -0.211269  0.009446       7        1   6.294401
0     -0.051644  0.039196       8        1   5.526343
5     -0.108780  0.026404       9        1   4.308225, topic_info=               Term        Freq       Total Category  logprob  loglift
2780       question  143.000000  143.000000  Default  30.0000  30.0000
1430       governor  191.000000  191.000000  Default  29.0000  29.0000
2346        nursing   86.000000   86.000000  Default  28.0000  28.0000
2788      questions   66.000000   66.000000  Default  27.0000  27.0000
2348  nursing homes   53.000000   53.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
2425        outside    6.204336   19.909124   Topic9  -5.5128   1.9787
418            care    6.910931   41.094037   Topic9  -5.4050   1.3619
2373           open    7.140826   78.709494   Topic9  -5.3723   0.7447
533          closed    6.340244   32.020465   Topic9  -5.4912   1.5252
2225            mta    5.646363   21.536824   Topic9  -5.6071   1.8059

[582 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
4         7  0.920467    absentee
6         1  0.206320  absolutely
6         8  0.618960  absolutely
8         8  0.753180       abuse
46        4  0.151718    adequate
...     ...       ...         ...
3950      2  0.890964   zip codes
3953      1  0.658731      zucker
3953      2  0.138680      zucker
3953      4  0.104010      zucker
3953      6  0.069340      zucker

[1594 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 2, 7, 3, 4, 8, 1, 6])